# Exploring Topic and Word Relationships

Here we explore different ways that the the newstest 2015 dataset can be looked at.

In [1]:
import math

def read_file(file_path, max_amount=math.inf):
    """
    Reads a number of lines from file and return list of lines
    :param file_path:
    :param max_amount:
    :return: list[str]
    """
    line_list = []

    with open(file=file_path) as f:
        curr_idx = 0
        while curr_idx < max_amount:
            line_list.append(f.readline())
            curr_idx += 1

    return line_list

print("Loading file")
l_raw = read_file('./trg.shuf', 5000000)
print("Finished loading in file")

Loading file
Finished loading in file


In [12]:
import math
import pandas as pd
from collections import Counter
import operator
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
import gensim
from gensim import corpora





def process_word(word):
    """
    Returns process word
    :param word:
    :return: str word
    """
    # lower
    word = word.lower()
    
    # Remove symbols
    if len(word) < 3:
        return None
    
    # Get morph
    t_word = wn.morphy(word)
    if t_word is not None:
        word = t_word

    # Get lemma
    word = WordNetLemmatizer().lemmatize(word)
    
    """
    # Remove stop
    if word in en_stop:
        return None
    """
    return word


def process_bpe(line_list, seq="@@", rm_words=["\n"]):
    """
    Transforms line list form read_file to list[list[str=words]], and bpe words merged together
    :param line_list:
    :param seq:
    :param rm_words:
    :return: ist[list[str=words]]
    """
    full_word_list = []
    for line in line_list:
        temp_words = line.split()
        full_words = []
        idx = 0
        while idx < len(temp_words):
            if temp_words[idx].endswith(seq):
                temp_str = ""
                while temp_words[idx].endswith(seq):
                    temp_str += temp_words[idx][:-(len(seq))]
                    idx += 1
                temp_str += temp_words[idx]
                if temp_str not in rm_words:
                    w = process_word(temp_str)
                    if w is not None:
                        full_words.append(w)
                idx += 1
            else:
                if temp_words[idx] not in rm_words:
                    w = process_word(temp_words[idx])
                    if w is not None:
                        full_words.append(w)
                idx += 1
        full_word_list.append(full_words)
    return full_word_list


def get_word_count(word_list):
    """
    Returns word count.
    :param word_list:
    :return: List[(str=word, int=count)]
    """
    word_count = {}
    for line in word_list:
        for word, val in Counter(line).most_common():
            if word in word_count:
                word_count[word] += val
            else:
                word_count[word] = val

    return sorted(word_count.items(), key=operator.itemgetter(1))


def get_topics_lda(word_list, num_topics=10):
    dictionary = corpora.Dictionary(word_list)
    corpus = [dictionary.doc2bow(text) for text in word_list]

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    topics = ldamodel.print_topics(num_words=10)
    for topic in topics:
        print(topic)

    return topics, ldamodel, corpus, dictionary

import random
l = random.sample(l_raw, 50000)

wl = process_bpe(line_list=l)
print(get_word_count(word_list=wl))
t, ldamodel,_, dictionary = get_topics_lda(word_list=wl)




[nltk_data] Downloading package wordnet to /home/nikita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nikita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('naismith', 1), ('bankrupcty', 1), ('one-to-one', 1), ('fuwa', 1), ('fuwa福娃', 1), ('beibei', 1), ('jingjing', 1), ('huanhuan', 1), ('yingying', 1), ('nini', 1), ('blowhole', 1), ('2379', 1), ('jillani', 1), ('dapper', 1), ('chaudhry', 1), ('manchin', 1), ('eur144.2', 1), ('eur190.0', 1), ('217,841', 1), ('ofac', 1), ('energy-policy', 1), ('6892', 1), ('benefit-sharing', 1), ('abs-related', 1), ('benazir', 1), ('bhutto', 1), ('49-211-4541092', 1), ('nuttiness', 1), ('gunfire', 1), ('crackle', 1), ('al-omari', 1), ('deraa', 1), ('23604', 1), ('90924', 1), ('icty', 1), ('2015-05-01', 1), ('toremar', 1), ('mobylines', 1), ('indoctrinate', 1), ('638', 1), ('alexis', 1), ('millerberg', 1), ('02261', 1), ('22335', 1), ('power-line', 1), ('mysteriously', 1), ('trumpeter', 1), ('cadillac', 1), ('ellington', 1), ('hotel-type', 1), ('dodelicious', 1), ('opfer', 1), ('spieler', 1), ('411-420', 1), ('obwohl', 1), ('eingetragen', 1), ('hattest', 1), ('habe', 1), ('3727', 1), ('psn', 1), ('scorpiog

(0, '0.022*"page" + 0.021*"wa" + 0.020*"new" + 0.019*"second" + 0.016*"this" + 0.010*"august" + 0.010*"july" + 0.009*"generate" + 0.008*"music" + 0.008*"speed"')
(1, '0.041*"you" + 0.033*"and" + 0.032*"the" + 0.024*"that" + 0.017*"are" + 0.017*"have" + 0.017*"not" + 0.017*"your" + 0.017*"for" + 0.015*"can"')
(2, '0.066*"the" + 0.029*"and" + 0.025*"wa" + 0.022*"his" + 0.015*"be" + 0.012*"year" + 0.012*"have" + 0.011*"say" + 0.009*"after" + 0.009*"from"')
(3, '0.021*"hotel" + 0.020*"2009" + 0.020*"room" + 0.015*"visit" + 0.012*"guest" + 0.011*"number" + 0.008*"breakfast" + 0.006*"italy" + 0.005*"upload" + 0.005*"spain"')
(4, '0.016*"date" + 0.013*"contact" + 0.011*"2005" + 0.011*"january" + 0.011*"join" + 0.010*"internet" + 0.009*"tel" + 0.008*"news" + 0.007*"comment" + 0.007*"100"')
(5, '0.043*"2016" + 0.036*"http" + 0.026*"post" + 0.022*"2015" + 0.021*"may" + 0.020*"currently" + 0.019*"2010" + 0.016*"2011" + 0.014*"2013" + 0.013*"2012"')
(6, '0.038*"night" + 0.029*"sat" + 0.021*"time" 




# Visualization
Here we visualize with pyLDAvis in an interactive mode the LDA analysis.

In [14]:
import pyLDAvis.gensim
from gensim import corpora
dictionary = corpora.Dictionary(wl)
corpus = [dictionary.doc2bow(text) for text in wl]
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

# Vocab Splitting

Here we get the vocab for each topic.

In [15]:
def get_representatives_lda(word_list, lda_model, dictionary, num_topics=10):
    max_line = [[]] * num_topics
    max_class = [0] * num_topics 
    class_vocab = {i:{} for i in range(0, num_topics)}
    
    for line in word_list:
        new_doc_bow = dictionary.doc2bow(line)
        s = ldamodel.get_document_topics(new_doc_bow)
        s.sort(key=lambda x: x[1])
        t_max_class, t_max_perc = s[-1]
        
        for word in line:
            if word in class_vocab[t_max_class]:
                class_vocab[t_max_class][word] += 1
            else:
                class_vocab[t_max_class][word] = 1
        
        if max_class[t_max_class] < t_max_perc:
            max_class[t_max_class] = t_max_perc
            max_line[t_max_class] = line
    
    return max_line, max_class, class_vocab

# max_l, max_c, class_vocab = get_representatives_lda(wl, ldamodel, dictionary)
"""
for idx in range(len(max_l)):
    print("\n")
    print("\n")
    print(max_c[idx])
    print(max_l[idx])
"""  

# get overlap
num_topics = 10

import numpy as np


def get_overlap_matrix(class_vocab, num_topics):
    over_lap_matrix = np.ones((num_topics, num_topics))

    for c in range(0, num_topics):
        for c2 in range(0, num_topics):
            # Skip self
            if c2 == c:
                continue
            # overlap
            words_in_c2_and_c1 = 0
            total_words_c2 = 0
            for word in class_vocab[c2]:
                total_words_c2 += 1  # class_vocab[c2][word]
                if word in class_vocab[c]:
                    words_in_c2_and_c1 += 1
            c_factor = len(class_vocab[c])/(len(class_vocab[c2]) + len(class_vocab[c]))
            c2_factor = len(class_vocab[c2])/(len(class_vocab[c2]) + len(class_vocab[c]))
            over_lap_matrix[c, c2] = (words_in_c2_and_c1/len(class_vocab[c2])) * c2_factor + (words_in_c2_and_c1/len(class_vocab[c])) * c_factor
    
    return over_lap_matrix


over_lap_matrix = get_overlap_matrix(get_representatives_lda(wl, ldamodel, dictionary, num_topics=num_topics)[2], num_topics)

print("Overlap Matrix of Vocab")
print(pd.DataFrame(over_lap_matrix))
print("\n")

print("Average for each topic")
# Remove diagonal as it skews data
print(pd.DataFrame(np.average(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1), axis=1)))
print("\n")

print("Average overall")
print(np.average(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1)))

print("\n")
print("Max")
print(np.max(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1)))
                

Overlap Matrix of Vocab
          0         1         2         3         4         5         6  \
0  1.000000  0.153552  0.170620  0.137132  0.147144  0.126164  0.130316   
1  0.153552  1.000000  0.421958  0.171800  0.157962  0.110282  0.131817   
2  0.170620  0.421958  1.000000  0.183536  0.167832  0.120024  0.144035   
3  0.137132  0.171800  0.183536  1.000000  0.139917  0.115452  0.128201   
4  0.147144  0.157962  0.167832  0.139917  1.000000  0.124566  0.162698   
5  0.126164  0.110282  0.120024  0.115452  0.124566  1.000000  0.122253   
6  0.130316  0.131817  0.144035  0.128201  0.162698  0.122253  1.000000   
7  0.165188  0.177800  0.200017  0.157406  0.182054  0.129697  0.162357   
8  0.148836  0.174395  0.184623  0.147097  0.181317  0.123303  0.159966   
9  0.095322  0.368113  0.312021  0.112646  0.086255  0.069065  0.072776   

          7         8         9  
0  0.165188  0.148836  0.095322  
1  0.177800  0.174395  0.368113  
2  0.200017  0.184623  0.312021  
3  0.157406  0